In [ ]:
from typing import Any, Dict, List, Optional
import random
import math
import torch.nn as nn
import torch 
import torch.nn.functional as F
from PIL import Image
import torchvision.transforms as transforms

class FocalLoss(nn.Module):
    """
    Wraps focal loss around existing loss_fcn(), i.e. criteria = FocalLoss(nn.BCEWithLogitsLoss(), gamma=1.5).
    """

    def __init__(self, gamma: float = 1.5, alpha: float = 0.25):
        """Initialize FocalLoss class with focusing and balancing parameters."""
        super().__init__()
        self.gamma = gamma
        self.alpha = torch.tensor(alpha)

    def forward(self, pred: torch.Tensor, label: torch.Tensor) -> torch.Tensor:
        """Calculate focal loss with modulating factors for class imbalance."""
        loss = F.binary_cross_entropy_with_logits(pred, label, reduction="none")
        pred_prob = pred.sigmoid()  # prob from logits
        p_t = label * pred_prob + (1 - label) * (1 - pred_prob)
        modulating_factor = (1.0 - p_t) ** self.gamma
        loss *= modulating_factor
        if (self.alpha > 0).any():
            self.alpha = self.alpha.to(device=pred.device, dtype=pred.dtype)
            alpha_factor = label * self.alpha + (1 - label) * (1 - self.alpha)
            loss *= alpha_factor
        return loss.mean(1).sum()

class HeatmapTrainer:  # BaseTrainerを継承しない
    def __init__(self, model, loss_fn):
        self.model = model
        self.loss_fn = loss_fn
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.model.to(self.device)
        
    def preprocess_batch(self, batch: Dict, img_size: tuple = (640, 640)) -> Dict:
        """
        Preprocess a batch of images by scaling and converting to float.
        
        Args:
            batch (Dict): Dictionary containing batch data with 'img' tensor.
            img_size (tuple): Target size (resz_w, resz_h) for resizing.
        
        Returns:
            (Dict): Preprocessed batch with resized images.
        """
        resz_w, resz_h = img_size
        
        # torch.tensorをデバイスに移動
        batch["img"] = batch["img"].to(self.device, non_blocking=True).float()
        
        # バッチサイズ、チャネル、高さ、幅を取得
        batch_size, channels, img_h, img_w = batch["img"].shape
        
        # アスペクト比を保持してリサイズするためのスケール計算
        scale = min(resz_w / img_w, resz_h / img_h)
        new_w = int(img_w * scale)
        new_h = int(img_h * scale)
        
        # リサイズ
        resized_imgs = F.interpolate(batch["img"], size=(new_h, new_w), mode='bilinear', align_corners=False)
        
        # パディング計算
        pad_w = resz_w - new_w
        pad_h = resz_h - new_h
        pad_left = pad_w // 2
        pad_right = pad_w - pad_left
        pad_top = pad_h // 2
        pad_bottom = pad_h - pad_top
        
        # パディング適用 (left, right, top, bottom)
        padded_imgs = F.pad(resized_imgs, (pad_left, pad_right, pad_top, pad_bottom), mode='constant', value=0)
        
        batch["img"] = padded_imgs
        
        return batch
        
    def forward_and_loss(self, batch, targets=None):
        # モデルを学習モードに設定
        self.model.model.train()
        
        # 1. 画像の推論（生の出力を取得）
        # YOLOモデルの内部モデル（nn.Module）に直接アクセス
        predictions = self.model.model(batch["img"])
        
        print(f"Raw predictions type: {type(predictions)}")
        print(f"Raw predictions length: {len(predictions)}")
        for i, pred in enumerate(predictions):
            print(f"Prediction {i} shape: {pred.shape}")
        
        # 2. ヒートマップ予測とロス計算（targetsがある場合）
        if targets is not None:
            loss = self.loss_fn(predictions, targets)
            return predictions, loss
        else:
            return predictions, None

# テスト用コードも修正
if __name__ == "__main__":
    import sys
    import os

    # ローカルのultralyticsパスを最優先に設定
    local_ultralytics_path = r"C:\Users\kotat\MyPrograms\MyKuzushiji\ultralytics"
    if local_ultralytics_path not in sys.path:
        sys.path.insert(0, local_ultralytics_path)

    # この後でインポート
    from ultralytics import YOLO

    import ultralytics, os
    print(f"Ultralytics path: {ultralytics.__file__}")
    print(f"Current working directory: {os.getcwd()}")
    
    try:
        # YAMLファイルからモデルを作成
        print("Creating model from YAML...")
        model = YOLO("yolov8n.yaml") 
        print("Model created successfully!")
        
        # モデル構造を確認
        print(f"Model head type: {type(model.model.model[-1])}")
        
        fl = FocalLoss(gamma=2.0, alpha=0.25)
        
        # PIL ImageをTensorに変換
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        
        heat_obj = HeatmapTrainer(model=model, loss_fn=fl)


        image = Image.open("ultralytics/models/yolo/detect/100241706_sep_100241706_00002_1.jpg")
        image_tensor = transform(image).unsqueeze(0)  # バッチ次元を追加
        print(f"Image tensor shape: {image_tensor.shape}")
        result = heat_obj.preprocess_batch({"img": image_tensor})
        print(f"Preprocessed batch shape: {result['img'].shape}")
        # 生の予測結果を取得
        print("Running forward pass...")
        predictions, _ = heat_obj.forward_and_loss(result)
        
        # 各予測の詳細を表示
        from matplotlib import pyplot as plt
        for i, pred in enumerate(predictions):
            print(f"Level {i}: {pred.shape}")
            for i in range(4):
                plt.subplot(1, 4, i + 1)
                plt.imshow(pred[0, i].cpu().detach().numpy(), cmap='hot')
            plt.show()
            
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback
        traceback.print_exc()

# # テスト用コード
# if __name__ == "__main__":
#     from ultralytics import YOLO
    
#     model = YOLO("yolov8n.pt")
#     fl = FocalLoss(gamma=2.0, alpha=0.25)
#     image = Image.open("ultralytics/models/yolo/detect/100241706_sep_100241706_00002_1.jpg")
    
#     # PIL ImageをTensorに変換
#     transform = transforms.Compose([
#         transforms.ToTensor(),
#     ])
#     image_tensor = transform(image).unsqueeze(0)  # バッチ次元を追加
    
#     heat_obj = HeatmapTrainer(model=model, loss_fn=fl)
#     result = heat_obj.preprocess_batch({"img": image_tensor})
#     print(f"Preprocessed batch shape: {result['img'].shape}")

#     pred = model(result["img"])
#     print(type(pred[0]))

#     # print(torch.tensor(pred).shape)

Ultralytics path: C:\Users\kotat\MyPrograms\MyKuzushiji\ultralytics\ultralytics\__init__.py
Current working directory: c:\Users\kotat\MyPrograms\MyKuzushiji\Craft_respect\test\Yolo_trial
Creating model from YAML...
Model created successfully!
Model head type: <class 'ultralytics.nn.modules.head.Detect'>
Error occurred: [Errno 2] No such file or directory: 'C:\\Users\\kotat\\MyPrograms\\MyKuzushiji\\Craft_respect\\test\\Yolo_trial\\ultralytics\\models\\yolo\\detect\\100241706_sep_100241706_00002_1.jpg'


Traceback (most recent call last):
  File "C:\Users\kotat\AppData\Local\Temp\ipykernel_22152\2099966471.py", line 231, in <module>
    image = Image.open("ultralytics/models/yolo/detect/100241706_sep_100241706_00002_1.jpg")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kotat\AppData\Local\anaconda3\envs\kuzushiji\Lib\site-packages\PIL\Image.py", line 3277, in open
    fp = builtins.open(filename, "rb")
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\kotat\\MyPrograms\\MyKuzushiji\\Craft_respect\\test\\Yolo_trial\\ultralytics\\models\\yolo\\detect\\100241706_sep_100241706_00002_1.jpg'
